If we want to compare to other methods we need to interpolate data back to regular grid. To be able to interpolate back to regular data, we need to create a .nc file that we can give to cdo.

Only process one dataset folder at a time.

Doesn't work for precip weighted at the moment.

In [26]:
# pylint: disable=E1101,R,C
import numpy as np
import gzip
import netCDF4 as nc
import matplotlib.pyplot as plt
import pickle
from braceexpand import braceexpand
import numpy as np
import os
import glob

In [27]:
DIRECTORY_DATASETS_INTERPOLATED = "Datasets/Interpolated/"
DIRECTORY_DATASETS_ORIGINAL = "Datasets/Original/"
DIRECTORY_IMAGES = "Images/"
DIRECTORY_SCRIPTS = "Scripts/"
DIRECTORY_OUTPUTS = "Output/Compare_UNet_architectures/"

In [28]:
def check_dict_conditions(dic, conditions):
    """
    Test whether dict "dic" fullfills the given conditions "conditions". 
    The latter are given as a array of key-value pairs.
    """
    for key, value in conditions:
        if key in dic.keys():
            if not dic[key] == value:
                return False
        else:
            return False
    return True

def braced_glob(path):
    l = []
    for x in braceexpand(path):
        l.extend(glob.glob(x))
    return l

In [29]:
# dataset parameters. To leave unspecified use: "*"
PREFIX = "HadCM3-flat"#{-precip*,}"
DO_SHUFFLE = False
ALL_VARIABLES = np.sort(["temp_1","precip","dO18"])#np.sort(["temp_1","precip","dO18"])  # "*" get all possible combinations 
DSET_NR = "1"
# helping vars
shuffle_dict = {True:"shuffle", False:"no-shuffle", "*": "*"}

# wildcards can be used in this filename.
DATASET_FOLDER = "{}_{}_{}_".format(PREFIX, shuffle_dict[DO_SHUFFLE], DSET_NR)#shuffle_dict[DO_SHUFFLE]
DATASET_FOLDER = DATASET_FOLDER + "-".join(ALL_VARIABLES) # using "*" + "-".join(ALL_VARIABLES) instead of DATASET_FOLDER + ... makes it possible to use ico folders too.
DATASET_FOLDER = os.path.join(DIRECTORY_OUTPUTS, DATASET_FOLDER)


dataset_description_files = []  # list to collect the paths to the configuration files
dataset_folders = []  # list to collect all the matching folders.
folder_exists = False

if not os.path.exists(DATASET_FOLDER):
    raise OSError("There exists no folder for the given specifications")

    
DATASET_DESCRIPTION_FILE = os.path.join(DATASET_FOLDER, "dataset-description.gz")

with gzip.open(DATASET_DESCRIPTION_FILE, 'rb') as f:
    dataset_description = pickle.load(f)


model_descriptions = []
testset_predictions = []
model_descriptions_paths = []

# specify the conditions that we want the runs to match
conditions = []# [()"NUM_EPOCHS",3)]


subdirs = [d for d in os.listdir(DATASET_FOLDER) if os.path.isdir(os.path.join(DATASET_FOLDER, d))]

for subdir in subdirs:
    files = [f for f in os.listdir(os.path.join(DATASET_FOLDER, subdir)) if os.path.isfile(os.path.join(DATASET_FOLDER, subdir, f))]   
    if "model_training_description.gz" in files and "predictions.gz" in files:        
        # this is a valid description of a model run. Store path and print description
        with gzip.open(os.path.join(DATASET_FOLDER, subdir, "model_training_description.gz"), 'rb') as f:
            tmp_description = pickle.load(f)
            if check_dict_conditions(tmp_description, conditions):  # check if the description satisfies our conditions
                model_descriptions.append(tmp_description)      
                model_descriptions_paths.append(os.path.join(DATASET_FOLDER, subdir))
                print("Path: ", subdir)
                for key, value in model_descriptions[-1].items():
                    print(key,": ", value)
                print("\n")

                with gzip.open(os.path.join(DATASET_FOLDER, subdir, "predictions.gz"), 'rb') as g:
                    testset_predictions.append(pickle.load(g))       
    else:
        print("Encountered Invalid directory")

Path:  -0x5e008a71f7450afc
MODELTYPE :  UNet_flat
DEPTH :  3
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
USE_CYLINDRICAL_PADDING :  False
USE_COORD_CONV :  False
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS :  ('Pixelwise',)
loss :  MSELoss
optimizer :  Adam
RUN_NR :  6
#params :  315809


Path:  -0x6cfcd55c64e6f28f
MODELTYPE :  UNet_flat
DEPTH :  3
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
USE_CYLINDRICAL_PADDING :  True
USE_COORD_CONV :  True
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS 

Path:  -0x1d3e5b0d85482bc3
MODELTYPE :  UNet_flat
DEPTH :  3
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
USE_CYLINDRICAL_PADDING :  False
USE_COORD_CONV :  False
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS :  ('Pixelwise',)
loss :  MSELoss
optimizer :  Adam
RUN_NR :  3
#params :  315809


Path:  0x166c4e94b8f4cd1d
MODELTYPE :  UNet_flat
DEPTH :  3
NUM_EPOCHS :  early_stopping
BATCH_SIZE :  8
LEARNING_RATE :  0.005
IN_CHANNELS :  2
CHANNELS_FIRST_CONV :  32
OUT_CHANNELS :  1
FMAPS :  (32, 32, 64, 64)
USE_CYLINDRICAL_PADDING :  True
USE_COORD_CONV :  True
ACTIVATION :  <class 'torch.nn.modules.activation.ReLU'>
NORMALIZATION :  <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
S_MODE_PREDICTORS :  ('Pixelwise', 'Pixelwise')
S_MODE_TARGETS :

# Step 1: Get required ground truth
(Requires grid_description only)

In [30]:
print("Target variable:", dataset_description["target_variables"])

required_files = []

if dataset_description["target_variables"][0] == "dO18":
    for filename in dataset_description["files_used"]:
        if "isotopes" in filename:
            required_files.append(filename)
else:
    raise NotImplementedError("Currently only d18O is a valid target variable.")

Target variable: ['dO18']


In [31]:
# names of datasets to which we want not to be missing at any timestep.
dnames = [DIRECTORY_DATASETS_ORIGINAL+"xnapa_isotopes.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_precip.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_slp.nc",\
          DIRECTORY_DATASETS_ORIGINAL+"xnapa_temp.nc"]

def get_shared_timesteps(dataset_names):
    """
    Not all datasets share the same timesteps. The biggest problems occur in the slp dataset. We want to exclude all
    time steps where one of the variables is missing
    """
    
    # get indices of elements that are shared for all variables.
    from functools import reduce
    ts = tuple([nc.Dataset(dataset_name,"a").variables["t"][:].data for dataset_name in dataset_names])
    common_dates = reduce(np.intersect1d, ts)
    
    return common_dates

In [32]:
isotopes_dset = nc.Dataset(required_files[0], "a")

lat = isotopes_dset.variables["latitude"][:].data[1:-1]
lon = isotopes_dset.variables["longitude"][:]

t = isotopes_dset.variables["t"][:].data
t_bnds = isotopes_dset.variables["t_bnds"][:]

c_dates = get_shared_timesteps(dnames)
# get the corresponding indices:
indices = []
for j, t_ in enumerate(isotopes_dset.variables["t"][:].data):
    if t_ in c_dates:
        indices.append(j)
indices = np.array(indices)
index_mask = np.logical_and(isotopes_dset.variables["t"][indices].data // 360 >= 654, \
                            isotopes_dset.variables["t"][indices].data // 360 < 1654)    
indices = indices[index_mask]  

d18O = np.squeeze(isotopes_dset.variables["dO18"][:].data)[indices,1:-1]

In [33]:
d18O_train = d18O[dataset_description["indices_train"],...]
d18O_test = d18O[dataset_description["indices_test"],...]

t_train = t[dataset_description["indices_train"],...]
t_test = t[dataset_description["indices_test"],...]

t_bnds_train = t_bnds[dataset_description["indices_train"],...]
t_bnds_test = t_bnds[dataset_description["indices_test"],...]

## 1) undo the standardization

In [34]:
# in order to compare with the original dataset 
# we need to rescale the prediction based on the standardization that we used in the model

rescaled_predictions = []

for i, description in enumerate(model_descriptions):
    rescaled_predictions.append([])
    for j, mode in enumerate(description["S_MODE_TARGETS"]):
        if mode == "Global":
            std = np.mean(np.std(d18O_train, axis=(0), keepdims=True), axis=(1,2), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_train, axis=(0,1,2), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)             
        elif mode == "Pixelwise":        
            std = np.std(d18O_train, axis=(0), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_train, axis=(0), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)   
        elif mode == "Global_mean_pixelwise_std":
            std = np.mean(np.std(d18O_train, axis=(0), keepdims=True), axis=(1,2), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_train, axis=(0), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)           
        elif mode == "Pixelwise_mean_global_std":
            std = np.std(d18O_train, axis=(0), keepdims=True)
            std[std==0] = 1
            mean = np.mean(d18O_train, axis=(0,1,2), keepdims=True)
            rescaled_predictions[-1].append((testset_predictions[i]["predictions"][:,j,...] * std) + mean)  
        elif mode == "None":
            rescaled_predictions[-1].append(testset_predictions[i]["predictions"])
        else:
            raise NotImplementedError("{} is not a valid keyword for standardization".format(mode))
    rescaled_predictions[-1] = np.stack(rescaled_predictions[-1],axis=1)

In [35]:
tocopy = ['longitude', 'latitude',]
dimscopy =['t', 'bnds', 'longitude', 'latitude']



for i, model_description_path in enumerate(model_descriptions_paths):
    for filename in required_files:
        original_dimensions  = nc.Dataset(filename).variables["dO18"].dimensions
        necessary_dimensions = (original_dimensions[0], original_dimensions[2], original_dimensions[3])
        original_dataype     = nc.Dataset(filename).variables["dO18"].datatype

        
        netcdf4_path = os.path.splitext(model_description_path)[0]+"_flat.nc"

        src = nc.Dataset(filename)
        dst = nc.Dataset(netcdf4_path, "w")
        # copy global attributes all at once via dictionary
        dst.setncatts(src.__dict__)
        # copy dimensions
        for name, dimension in src.dimensions.items():
            if name in dimscopy:
                dst.createDimension(name, (len(dimension) if not dimension.isunlimited() else None))
        # copy all file data except for the excluded
        for name, variable in src.variables.items():
            if name in tocopy:
                x = dst.createVariable(name, variable.datatype, variable.dimensions)
                dst[name][:] = src[name][:]
                # copy variable attributes all at once via dictionary
                dst[name].setncatts(src[name].__dict__)

        target_var_attribute_dict = nc.Dataset(filename).variables["dO18"].__dict__
        dst.createVariable("dO18", original_dataype, necessary_dimensions)
        dst.variables["dO18"].setncatts(target_var_attribute_dict)
        dst.createVariable("t", "float64", ("t"))
        dst.createVariable("t_bnds", "float64", ("t","bnds"))
        dst.variables["t"][:] = t_test
        dst.variables["t_bnds"][:] = t_bnds_test
        tmp = np.pad(np.squeeze(rescaled_predictions[i]),((0,0),(1,1),(0,0)), 'constant', \
                     constant_values=target_var_attribute_dict["missing_value"])
        dst.variables["dO18"][:] = tmp

        # print(dst.)
        dst.close()
        src.close()